In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle


* Reading data

In [6]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


* Preprocessing the data

* Dropping irrelevant columns

In [7]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [8]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


* Encoding Categorical Variables

In [9]:
labelencoder_gender=LabelEncoder()
df['Gender']=labelencoder_gender.fit_transform(df['Gender'])


In [10]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [11]:
df['Gender'].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [12]:
# one hot encode geography column using one hot encoder 
from sklearn.preprocessing import OneHotEncoder
onehotencoder_geo=OneHotEncoder()
geography_encoded=onehotencoder_geo.fit_transform(df['Geography'].values.reshape(-1,1))



In [13]:
geo_encoded_df = pd.DataFrame(geography_encoded.toarray(), columns=onehotencoder_geo.get_feature_names_out(['Geography']))

In [14]:
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [15]:
df = pd.concat([df.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [16]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [17]:
# saving the encoders into a pickel file
with open('labelencoder_gender.pkl', 'wb') as file:
    pickle.dump(labelencoder_gender, file)

with open('onehotencoder_geo.pkl', 'wb') as file:
    pickle.dump(onehotencoder_geo, file)

In [18]:
# Dividing the data into independent and dependent variables
X=df.drop('Exited', axis=1)
y=df['Exited']

In [19]:
# splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Scaling the data
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [21]:
# Saving the scaler into a pickle file
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

* Model Creation

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


In [23]:
# Building the model
model=Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
opti = tf.keras.optimizers.Adam(learning_rate=0.01) # defining the optimizer
loss = tf.keras.losses.BinaryCrossentropy() # defining the loss function
model.compile(optimizer=opti, loss=loss, metrics=['accuracy'])

In [26]:
# create a tensorboard directory
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


In [27]:
# Set up early stopping
early_stop = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [28]:
# Training the model
history=model.fit(X_train, y_train, epochs=100,validation_data=(X_test, y_test), callbacks=[early_stop, tensorboard_callback])

Epoch 1/100


250/250 [==============================] - 3s 4ms/step - loss: 0.4030 - accuracy: 0.8298 - val_loss: 0.3713 - val_accuracy: 0.8465
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3546 - accuracy: 0.8540 - val_loss: 0.3505 - val_accuracy: 0.8570
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3490 - accuracy: 0.8599 - val_loss: 0.3544 - val_accuracy: 0.8595
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3456 - accuracy: 0.8583 - val_loss: 0.3475 - val_accuracy: 0.8620
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3428 - accuracy: 0.8627 - val_loss: 0.3414 - val_accuracy: 0.8550
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3381 - accuracy: 0.8620 - val_loss: 0.3513 - val_accuracy: 0.8605
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3353 - accuracy: 0.8630 - val_loss: 0.3424 - val_accuracy: 0.85

In [29]:
# Save the model
model.save('model.h5')

c:\Users\dscha\OneDrive\Desktop\Churn_Classification_Project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
